In [2]:
import numpy as np
import os
import pandas as pd
import requests
import json
import urllib
import tensorflow as tf
import tensorflow_datasets
from tensorflow import keras
from gensim.models.callbacks import CallbackAny2Vec
%config Completer.use_jedi =False
from gensim.models import Word2Vec

In [4]:
models = os.path.join(os.getcwd(),"Models")
if not os.path.exists(models):
    os.mkdir(models)

In [9]:
#using gensim to learn feature embeddings for each hero
'''Each hero has an embedding size of 30.
Trained using cbow'''
modified_names_array=np.load('picks_and_bans\\modified_names_array.npy')
modified_names_array=modified_names_array.tolist()
model=Word2Vec(sentences=modified_names_array,min_count=10,vector_size=30,workers=6,hs=0,negative=5,ns_exponent=0.75,sg=0)
model.save('Models\\modified_gensim_w2v_30')

In [6]:
model=Word2Vec.load('Models\\modified_gensim_w2v_30')

In [10]:
model.wv.most_similar('Void Spirit', topn=10)

[('Storm Spirit', 0.7895029783248901),
 ('Ember Spirit', 0.7521005272865295),
 ('Death Prophet', 0.6182236671447754),
 ('Kunkka', 0.5723374485969543),
 ('Queen of Pain', 0.5621659755706787),
 ('Invoker', 0.5509742498397827),
 ('Tiny', 0.5429650545120239),
 ('Puck', 0.5337207913398743),
 ('Doom', 0.5308213829994202),
 ('Leshrac', 0.4920659363269806)]

In [11]:
model.wv['Weaver']

array([-0.14890763,  0.12549475, -0.7875333 , -0.74437964,  0.2588019 ,
       -0.2920706 , -0.1750087 , -0.74716747, -0.0983211 , -0.9190139 ,
        1.4005284 ,  1.1487281 ,  0.98441565, -0.8690871 , -1.4844764 ,
        0.6120156 , -0.5402824 ,  0.7261612 ,  1.141535  ,  1.0398731 ,
        0.82617176, -0.8298969 , -0.6544508 , -0.27534065, -0.8296161 ,
       -2.3745613 , -0.15422688,  1.6325833 , -0.19644968, -0.60579926],
      dtype=float32)

In [9]:
#Replacing names with embeddings in names array
embedding_array=[]
for i in modified_names_array:
    buff=[]
    for j in i:
        buff.append(model.wv[j].tolist())
    embedding_array.append(buff)
embedding_array=np.array(embedding_array)
np.save("picks_and_bans\\modified_embedding_array_30",embedding_array)

In [13]:
#Creating a csv file with hero to embeddings mapping
embed_dict={}
hero_df=pd.read_csv('hero_df.csv')
hero_dict={}
for i in hero_df['sid']:
    hero_dict[i]=hero_df[hero_df['sid']==i]['localized_name'].values[0]
for i in hero_dict.values():
    embed_dict[i]=model.wv[i]
embedding_df=pd.DataFrame(embed_dict).T
embedding_df.to_csv('modified_hero_to_embedding.csv')